## no se observa la estructura normal descrita en la documentacion de Tweeter
- No se encuentra retweet_status 
- No se observa screen_name
- No se observa presencia de el tag entities

In [1]:
import os
import string
import pandas as pd

import json
from collections import Counter
import re
from glob import glob
from datetime import datetime
from matplotlib import pyplot as plt
from time import gmtime, mktime, strptime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# funcion que obtiene los elementos unicos a partir de una lista
def unique(lista_in):
 
    # inicializar la lista vacia
    unique_list = []
     
    # recorrer cada elemento en la lista
    for x in lista_in:
        # revisa si el elemento existe en la lista unica
        if x not in unique_list:
            unique_list.append(x)
    # print list
    return unique_list

## Load json file

In [3]:
DIR = 'C:/Users/gufernandez/Documents/tweets.json/farmers-protest-tweets-2021-2-4.json'

In [5]:
tweets_data = []

tweets_data_path = DIR
tweets_file = open(tweets_data_path, "r")

### read line by line

In [6]:
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [7]:
# Elimina posibles tweet sin user
for tweet in tweets_data:
    try:
        user = tweet['user']
    except:
        tweets_data.remove(tweet)
 
print(len(tweets_data))

117407


### Extraer info de importancia para la solución

In [8]:
rows_list = []
for tweet in tweets_data:
    
    author = ""
    rtauthor = ""
    
    ## Extrae la información de usuario autor y el retweet, si es un retweet
    try:
        
        author = tweet['user']['username']
        rtauthor = tweet['retweeted_status']['user']['username']
        
    except:

        ## Si no es un retweet obtiene la información del usuario autor solamente
        try:
            author = tweet['user']['username']
        except:
            continue
    
    # Se extrae la data desde el tweet
    create_date = datetime.strptime(str(tweet['user']['created'])[0:10], '%Y-%M-%d').date()
    followers = tweet['user']['followersCount']
    replyCount = tweet['replyCount']
    retweetedTweet = tweet['retweetedTweet']
    retweetCount = tweet['retweetCount']
    
    all_content_tweet = tweet['user']
    
    # fue necesario extraer los hashtag desde el content del tweet ya que no esta presente el tag de 'entities' donde se encuentra
    # la info de los hashtag de un tweet
    hashtags_list = re.split(r',|\n| ', str(tweet['content']))
    hashtags = []
    
    # se considerara hashtag todo aquel texto que comiece con el simbolo '#'
    for element in hashtags_list:
        
        regexp = re.compile(r'^#')
        
        if regexp.search(element):
            if str(element) == '[]':
                element = ''
            hashtags.append(str(element))
    
    ## Transforma la información en un diccionario para generar un dataframe nuevo
    dict1 = {}
    dict1.update({'author': author, 'create_date': create_date, 'replyCount': replyCount, 'hashtags':hashtags , 'retweetedTweet': retweetedTweet, 'retweetCount': retweetCount , 'followers': followers, 'tweet': all_content_tweet})
    rows_list.append(dict1)

In [9]:
tweets = pd.DataFrame(rows_list)

In [10]:
tweets.head(1)

,author,create_date,replyCount,hashtags,retweetedTweet,retweetCount,followers,tweet
0,ArjunSinghPanam,2009-01-06,0,"[#ModiDontSellFarmers, #FarmersProtest, #FreeN...",None,0,603,"{'username': 'ArjunSinghPanam', 'displayname':..."


### 1 .- Obtenga cuales fueron los 10 tweets más retweeted y a que usuarios pertenecen esos tweets.

In [11]:
top_ten_retweet_user = tweets[['author','retweetCount']].sort_values(by='retweetCount', ascending=False).head(10)

In [12]:
# Resultado esperado para objetivo_1
top_ten_retweet_user

,author,retweetCount
111329,RakeshTikaitBKU,7723
7645,dhruv_rathee,6164
89780,rupikaur_,4673
88911,amaanbali,3742
111556,jedijasmin_,3332
64492,rupikaur_,3230
108072,RaviSinghKA,3182
60721,sherryontopp,3057
29510,sherryontopp,3040
24160,sherryontopp,2622


### 2.-  Los top 10 users en función a la cantidad de tweets que emitieron (no considerar retweets).

In [13]:
top_ten_tweet_user = tweets[['author']].value_counts().head(10)

In [14]:
for i, row in top_ten_tweet_user.iteritems():
  print(str(i), row)

('jot__b',) 1019
('rebelpacifist',) 850
('MaanDee08215437',) 830
('Gurpreetd86',) 636
('GurmVicky',) 597
('shells_n_petals',) 576
('preetysaini321',) 573
('ish_kayy',) 515
('KaurDosanjh1979',) 512
('DigitalKisanBot',) 490


## 3.- Obtenga los 10 días donde hay más tweets, muestre el día, la cantidad de tweets y todos los hastags que se usaron ese día

In [15]:
top_ten_tweet_by_day = tweets['create_date'].value_counts().head(10)

In [16]:
# Extraigo el listado de dias que presentan la mayor cantidad de tweets
list_days = top_ten_tweet_by_day

In [18]:
list_hashtag = []
top_ten_tweets_by_day = []
hashtag_values = []

# loop sobre los 10 dias con mas tweets
for index, value in list_days.items():

    # obtengo los hashtags utilizados en el día especificado en el loop
    hashtags_on_date = tweets['hashtags'][(tweets['create_date']==index)]
    
    for hashtag_list_on_date in hashtags_on_date:
        hashtag_values += (hashtag_list_on_date)   
             
    final_list_hashtag = unique(hashtag_values)        
            
    top_ten_tweets_by_day.append([index, value,final_list_hashtag])


## 4.- Los top 10 hashtags más usados.

In [20]:
hashtags = tweets['hashtags']
hashtag_words = []

for hashtag_list in hashtags:
    hashtag_words += (hashtag_list_on_date)   

# genero la lista del top 10 de hashtags mas usados
c = Counter(hashtag_values)

In [21]:
print ("",c.most_common(10))    

 [('#FarmersProtest', 27863), ('#ReleaseDetainedFarmers', 1858), ('#MahapanchayatRevolution', 1560), ('#FarmersMakeIndia', 1472), ('#RepealOnlyWayAhead', 1465), ('#Pagdi_Sambhal_Jatta', 1083), ('#IndiaBeingSilenced', 1052), ('#शहीद_जवान_शहीद_किसान', 844), ('#farmersprotest', 824), ('#StandWithFarmers', 813)]


## 5.- Los top 10 users más influyentes en función de lo retweeted de sus tweets.

In [22]:
tweets.head()

,author,create_date,replyCount,hashtags,retweetedTweet,retweetCount,followers,tweet
0,ArjunSinghPanam,2009-01-06,0,"[#ModiDontSellFarmers, #FarmersProtest, #FreeN...",None,0,603,"{'username': 'ArjunSinghPanam', 'displayname':..."
1,PrdeepNain,2021-01-29,0,"[#FarmersProtest, #ModiIgnoringFarmersDeaths, ...",None,0,14,"{'username': 'PrdeepNain', 'displayname': 'Pra..."
2,parmarmaninder,2012-01-27,0,[#FarmersProtest],None,0,72,"{'username': 'parmarmaninder', 'displayname': ..."
3,anmoldhaliwal,2010-01-28,0,"[#farmersprotest, #NoFarmersNoFood]",None,0,51,"{'username': 'anmoldhaliwal', 'displayname': '..."
4,KotiaPreet,2021-01-08,0,"[#KisanEktaMorcha, #FarmersProtest, #NoFarmers...",None,0,5,"{'username': 'KotiaPreet', 'displayname': 'Pre..."


In [23]:
# Obtengo como dataframe el listado de 10 usuarios mas influyentes
sumOfretweetCount = tweets.groupby(by=['author'], as_index=False)['retweetCount'].sum().sort_values(by=['retweetCount'], ascending=False).head(10)

In [24]:
sumOfretweetCount

,author,retweetCount
16605,amaanbali,26354
24073,saahilmenghani,23288
12164,RaviSinghKA,22974
24724,sherryontopp,19175
11837,RakeshTikaitBKU,12001
24034,rupikaur_,11420
22538,news24tvchannel,10960
19566,iMani_KaurRai,10636
9532,Monica_Gill1,8593
17502,bhupenderc19,7360


## 6.- Guarde en un archivo avro todos los tweets para el hashtag #ReleaseDetainedFarmers entre el 2021-02-12 y 2021-02-24

In [52]:
## Paquetes requeridos
!pip install avro
!pip3 install rec-avro
!pip install fastavro

In [53]:
from fastavro import writer, reader, schema
from rec_avro import to_rec_avro_destructive, from_rec_avro_destructive, rec_avro_schema
#avro.schema.parse(str(tweets['tweet'][0]))

In [54]:
## Convierte jsonobj a avro_file
def turntoavro(data):
    avro_objects = [to_rec_avro_destructive(data)]
    with open('json_in_avro.avro', 'a+b') as f_out:
        writer(f_out, schema.parse_schema(rec_avro_schema()), avro_objects)

In [55]:
# Obtengo el subser de tweets que se encuentran entre el '2021-02-12' y '2021-02-24'
#### NO EXISTEN TWEETS EN EL INTERVALO INDICADO

subset_tweets_on_dates = tweets['tweet'][(pd.to_datetime(tweets['create_date'])>='2021-02-12') & (pd.to_datetime(tweets['create_date'])<='2021-02-24')]

In [56]:
try:
    for raw_tweet in subset_tweets_on_dates['tweet']:
        turntoavro(raw_tweet)
except:
    print("No existen tweets en el intervalo indicado\n")

No existen tweets en el intervalo indicado

